In [7]:
# 예시로 Jupyter Notebook에서 테스트하는 코드
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('../data/supplements_combined_20250311_2016.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40459 entries, 0 to 40458
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제조사     40459 non-null  object
 1   제품명     40459 non-null  object
 2   신고번호    40459 non-null  int64 
 3   등록일     40459 non-null  int64 
 4   유통기한    40457 non-null  object
 5   기능성     40451 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [11]:
# 결측값 제거
df = df.dropna(subset=['기능성'])

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['기능성'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 특정 키워드로 추천 제품 찾기
def recommend(keyword, limit=5):
    # 키워드에 포함된 제품 필터링
    matched_products = df[df['기능성'].str.contains(keyword, case=False, na=False)]
    
    if matched_products.empty:
        return []
    
    idx = matched_products.index[0]
    sim_scores = sorted(list(enumerate(cosine_sim[idx])), key=lambda x: x[1], reverse=True)[1:limit+1]
    product_indices = [i[0] for i in sim_scores]
    
    return df[['제품명', '제조사']].iloc[product_indices]

# 예시 테스트: '비타민 C'와 관련된 추천 제품
recommend_results = recommend('비타민 C', limit=5)
recommend_results

MemoryError: Unable to allocate 12.2 GiB for an array with shape (40451, 40451) and data type float64